<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/TF_Traffic_Sign_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Data Gathering

Download data from a given URL.

In [2]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
!unzip -q traffic-signs-data.zip

--2020-09-03 13:30:13--  https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 13.227.220.34, 13.227.220.60, 13.227.220.172, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|13.227.220.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  78.0MB/s    in 1.5s    

2020-09-03 13:30:14 (78.0 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]



Read data from pickle files.

In [3]:
import pickle

train_data = pickle.load(open("train.p", "rb"))
val_data = pickle.load(open("valid.p", "rb"))
test_data = pickle.load(open("test.p", "rb"))

In [4]:
get_features_labels = lambda x : (x["features"], x["labels"])
X_train, y_train = get_features_labels(train_data)
X_val, y_val = get_features_labels(val_data)
X_test, y_test = get_features_labels(test_data)

Since the pixel lies between 0-255, need **scaling** -> would add a rescaling layer later in the model.

In [5]:
print("Pixel range:", np.min(X_train), np.max(X_train))

Pixel range: 0 255


## Data Batchifying

Convert numpy dataset to tensorflow dataset.

In [6]:
convert_to_tfds = lambda x, y : tf.data.Dataset.from_tensor_slices((x, y))
ds_train = convert_to_tfds(X_train, y_train)
ds_val = convert_to_tfds(X_val, y_val)
ds_test = convert_to_tfds(X_test, y_test)

Shuffle for better generalization and batchify data.

In [7]:
BATCH_SIZE = 128
shuffle_batch = lambda ds, ds_size : ds.shuffle(ds_size).batch(BATCH_SIZE)
ds_train = shuffle_batch(ds_train, len(X_train))
ds_val = shuffle_batch(ds_val, len(X_val))
ds_test = shuffle_batch(ds_test, len(X_test))

## Model Architecture

In [8]:
num_train_samples, *input_shape = X_train.shape
num_classes = 43
print("Input shape:", input_shape)

Input shape: [32, 32, 3]


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [10]:
model = Sequential()

model.add(Rescaling(scale=1./255, input_shape=input_shape))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu")) # Input shape: (32 x 32 x 32)
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu")) # Input shape: (32 x 32 x 32)
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))                                            # Input shape: (16 x 16 x 32)

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu")) # Input shape: (16 x 16 x 64)
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu")) # Input shape: (16 x 16 x 64)
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))                                            # Input shape: (8 x 8 x 64)

model.add(Flatten())                                                                 # Input shape: (4096)
model.add(Dense(512, activation="relu"))                                             # Input shape: (512)
model.add(Dense(num_classes, activation="softmax"))                                  # Input shape: (num_classes)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [11]:
rmsprop_opt = tf.optimizers.RMSprop()
model.compile(loss="sparse_categorical_crossentropy", optimizer=rmsprop_opt, metrics=["acc"])

## Training

In [12]:
history = model.fit(ds_train, epochs=20, validation_data=ds_val)

Epoch 1/20
272/272 [==============================] - 4s 14ms/step - loss: 0.9632 - acc: 0.8099 - val_loss: 3.2432 - val_acc: 0.4009
Epoch 2/20
272/272 [==============================] - 3s 13ms/step - loss: 0.0662 - acc: 0.9835 - val_loss: 0.4437 - val_acc: 0.9039
Epoch 3/20
272/272 [==============================] - 3s 13ms/step - loss: 0.0363 - acc: 0.9919 - val_loss: 0.6682 - val_acc: 0.8948
Epoch 4/20
272/272 [==============================] - 3s 13ms/step - loss: 0.0260 - acc: 0.9945 - val_loss: 0.4250 - val_acc: 0.9397
Epoch 5/20
272/272 [==============================] - 3s 13ms/step - loss: 0.0174 - acc: 0.9961 - val_loss: 0.5060 - val_acc: 0.9361
Epoch 6/20
272/272 [==============================] - 3s 13ms/step - loss: 0.0133 - acc: 0.9971 - val_loss: 0.4059 - val_acc: 0.9619
Epoch 7/20
272/272 [==============================] - 3s 13ms/step - loss: 0.0130 - acc: 0.9972 - val_loss: 0.8667 - val_acc: 0.9420
Epoch 8/20
272/272 [==============================] - 4s 13ms/step - 